# NOTEBOOK PARA REALIZAR TODO EL ANÁLISIS DE SECUENCIACIÓN, ENSAMBLE Y ANÁLISIS DE GENOMAS DE VIRUS DE INFLUENZA



## LIMPIEZA DE ARCHIVOS FASTQ

Definir variables útiles

In [ ]:
MUESTRA=""
FASTQ1=""
FASTQ2=""

Cambiar el directorio a la carpeta a procesar. Ya debe contener los archivos FASTQ a procesar. <span style="font-size: 150%; color: red;"> **Modificar dirección si es necesario.**</span>

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

Crear un directorio para depositar el análisis de calidad de lecturas crudas.

Crear una carpeta para depositar las lecturas crudas. Ahí se moverán los archivos con las lecturas crudos.

In [ ]:
mkdir CALIDAD_CRUDA
mkdir RAW
mv CPA* ./RAW

Activar ambiente calidad

In [ ]:
conda activate calidad

Ejectuar análisis de calidad inicial

In [ ]:
fastqc ./RAW/${FASTQ1} ./RAW/${FASTQ2} --outdir ./CALIDAD_CRUDA --threads 20

Revisar los archivos HTML de salida para identificar parámetros que ayuden a la limpieza. Esto se puede lograr transfiriendo los archivos HTML a una computadora local y visualizando en un navedador de internet.


In [ ]:
# Descomentar y modificar si se realiza algun filtrado sobre los archivos crudos que cree otros archivos, para sobreescribir las variables con los archivos a usar:
# FASTQ1=""
# FASTQ2=""

Ejecutar el programa de limpieza. <span style="font-size: 150%; color: red;"> **Ajustar parámetros de acuerdo a las necesidades.**</span> Cuidado al modificar los cores (ver documentación). Los resultados se guardarán en ./TRIMMING/

In [ ]:
trim_galore --paired --retain_unpaired --gzip --fastqc --cores 3 --clip_R1 18 --clip_R2 18 --three_prime_clip_R1 4 --three_prime_clip_R2 16 --length 120 -o TRIMMING/ ./RAW/${FASTQ1} ./RAW/${FASTQ2}

Revisar los archivos HTML de salida para verificar los resultados de la limpieza. Queda a consideración de la experiencia del usuario el quedar conforme con la limpieza realizada.

## ALINEAMIENTO DE LECTURAS CONTRA BASE DE DATOS DE INFLUENZA

Hacer directorio para los alineamientos:

In [ ]:
mkdir -p ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING

Usar nombres de archivos originales para dar con los nombres de archivos producidos por trim_galore

In [ ]:
FASTQ1_P="${FASTQ1/_filtered.fastq.gz/_filtered_val_1.fq.gz}"
FASTQ2_P="${FASTQ2/_filtered.fastq.gz/_filtered_val_2.fq.gz}"
FASTQ1_U="${FASTQ1/_filtered.fastq.gz/_filtered_unpaired_1.fq.gz}"
FASTQ2_U="${FASTQ2/_filtered.fastq.gz/_filtered_unpaired_2.fq.gz}"

Crear ligas simbolicas a los archivos a usar:

In [ ]:
ln -s TRIMMING/${FASTQ1_P} reads_r1_tr
ln -s TRIMMING/${FASTQ2_P} reads_r2_tr
ln -s TRIMMING/${FASTQ1_U} reads_u1_tr
ln -s TRIMMING/${FASTQ2_U} reads_u2_tr

Activar ambiente conda alineamiento

In [ ]:
conda activate alineamiento

Hacer los alineamientos de las secuencias pareadas y no pareadas contra la base de datos viral preparada en la UASIP.

In [ ]:
#Alineamiento de secuencias pareadas 
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_r1_tr reads_r2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/paired.sam 

#Alineamiento de secuencias no pareadas de R1
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u1_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r1.sam

#Alineamiento de secuencias no pareadas de R2
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r2.sam

Entrar al directorio donde se depositaron los resultados del alineamiento, formatear los archivos sam resultantes en un archivo bam ordenado y regresar al directorio del proyecto.

In [ ]:
cd ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING 
samtools merge -u - paired.sam unpaired_r1.sam unpaired_r2.sam | samtools sort -o all_sorted.bam && rm paired.sam unpaired_r1.sam unpaired_r2.sam
cd -

Crear ocho directorios para depositar las lecturas por el segmento al cual alinearon.

In [ ]:
mkdir ALINEAMIENTO/BWA/S{1..8}

Ejecutar el siguiente programa. Este programa abre el archivo bam, elimina alineamientos con alineamientos secundarios, y obtiene las lecturas por cada segmento. Se analiza, por segmento, el formato CIGAR asociado de cada lectura. Se filtran aquellas secuencias que alinearon en al menos 70% de sus bases. Se guardan los nombres de las lecturas pertinentes y se extraen las secuencias de los archivos fastq limpios usando dichos nombres. Los archivos producidos se guardan en las ubicaciones pertinentes.

In [ ]:
~/analisis_influenza/bin/separar_seqs_segmentos.sh

## ENSAMBLE DE NOVO DE GENOMAS DE INFLUENZA

Activa una terminal tmux para que se encargue de enviar los procesos aunque salgamos del servidor

In [ ]:
tmux new -s ensamble

Pegar el siguiente código de instrucciones en la terminal tmux. <span style="font-size: 150%; color: red;"> **Ajustar parámetros de acuerdo a las necesidades.**</span> Cuidado de pegar con tamaño de letra pequeño, si no, temux cambia de orden algunas instrucciones. Este paso está diseñado para ejecutarse sobre todos los segmentos virales. Es importante revisar cada archivo log de cada subcarpeta para verificar que se haya ejecutado todos los procesos, desde el kmer 11 al kmer 127. Se hace en terminal tmux para que el ciclo no caiga cuando se termine la conexion con el servidor. En caso de que el servidor se caiga y evite que algunos segmentos terminen de ensamblarse, modificar el ciclo for y volver a correr desde el segmento que quedó incompleto.

In [ ]:
for i in {1..8}
do
    ID="S${i}"
    id="s${i}"
    LOG="./ENSAMBLE/SPADES/${ID}/log_11_127_${ID}"
    mkdir -p "./ENSAMBLE/SPADES/${ID}"

    TMP_SCRIPT=$(mktemp /tmp/run_${ID}_XXXXXX.sh)

    cat > "$TMP_SCRIPT" <<EOF
    #!/bin/bash

    echo "=== Iniciando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    ~/analisis_influenza/bin/ensamble.sh \\
        -a "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r1.fq.gz" \\
        -b "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r2.fq.gz" \\
        -x "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u1.fq.gz" \\
        -y "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u2.fq.gz" \\
        -t 28 --kini 11 --kstep 4 --kfin 127 -o ./ENSAMBLE/SPADES/${ID}/OUT_11_127 >> "$LOG" 2>&1

    echo "=== Finalizando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    rm -- "\$0"
EOF

    chmod +x "$TMP_SCRIPT"
    nohup "$TMP_SCRIPT" &
    wait
done

**Se puede cerrar sesion con Ctrl + B, y luego presionar D**. Para reconectar la terminal tmux usar: **tmux attach -t ensamble**

Continuar después de que el proceso anterior haya finalizado. Revisar que con ambiente alineamiento este activado y estar posicionados en la carpeta de trabajo.

In [ ]:
conda activate alineamiento
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

Hacer loop para unir todos los resultados de blast (si se hizo por partes) y analizar los resultados, extraer secuencias, revisar orientacion y calcular profundidad.

In [ ]:
for i in {1..8}; do
    ID="S${i}"

    cd "./ENSAMBLE/SPADES/${ID}"

    ~/analisis_influenza/bin/merge_blast_results.sh

    ~/analisis_influenza/bin/ensamble_analyze_blast.sh ${ID}_blastn-careful_merged.txt
    
    cd -
done

cd "./ENSAMBLE/SPADES/" && cat S*/best_result_S*_blastn-careful_merged.txt >all_segm_best_result_blast.txt && cd -

Ingresar al directorio SPADES para los siguientes analisis

In [ ]:
cd ./ENSAMBLE/SPADES

Obtener porcentajes de lecturas usadas por cada segmento

In [ ]:
~/analisis_influenza/bin/unir_profundidades.sh \
    --S1 S1/PROFUNDIDAD/S1_profundidad \
    --S2 S2/PROFUNDIDAD/S2_profundidad \
    --S3 S3/PROFUNDIDAD/S3_profundidad \
    --S4 S4/PROFUNDIDAD/S4_profundidad \
    --S5 S5/PROFUNDIDAD/S5_profundidad \
    --S6 S6/PROFUNDIDAD/S6_profundidad \
    --S7 S7/PROFUNDIDAD/S7_profundidad \
    --S8 S8/PROFUNDIDAD/S8_profundidad

~/analisis_influenza/bin/ensamble_cal_porc_lect_mapeadas.sh

Salir del ambiente alineamiento

In [ ]:
conda deactivate alineamiento

Activar ambiente R

In [ ]:
conda activate R

Graficar profundidades y uso de secuencias por segmento

In [ ]:
~/analisis_influenza/bin/graficar_profundidad.R --input_file profundidades_finales.tsv --muestra ${MUESTRA}
~/analisis_influenza/bin/graficar_lecturas.R --input_file uso_de_lecturas.tsv --muestra ${MUESTRA}

Desactivar ambiente R

In [ ]:
conda deactivate

## ANALISIS DE SUPTIFICACIÓN

Se debe hacer un análisis de subtipificación, donde se haga un conteo de los subtipos de cada subject alineado, y se obtenga una tabla con los mejores 5 subtipos.

In [ ]:
for i in {1..8}; do
    
    DIR_S="S${i}"
    cd ./${DIR_S}
    
    echo "Ejecutando análisis de subtipificación para ${DIR_S}"

    ~/analisis_influenza/bin/analisis_subtipificacion.sh best_result_${DIR_S}_sequence_corrected.fna
    
    echo " "

    cd -

    (
        echo "Análisis de subtipificacion para ${DIR_S}"
        cat ./${DIR_S}/${DIR_S}_subtipos_top5.tsv
        echo ""
    ) >> "analisis_subtipificacion.tsv"
done

## ANALISIS DE PATOGENICIDAD

Este análisis sólo se realiza sobre el S4. Cambiar de directorio.

In [ ]:
cd S4

Revisar que en alguno de los marcos de lectura ya corregidos no existan, o existan pocos, codones de paro.

In [ ]:
seqkit translate best_result_S4_sequence_corrected.fna -f1,2,3 -F 

Realizar el análisis de patogenicidad que buscará 4 o más residuos positivos entre el motivo PXnGLF (donde Xn son residuos positivos (K, H, R), principalmente). Se realiza sobre todos los marcos de lectura por si es necesario revisarlo.

In [ ]:
~/analisis_influenza/bin/analisis_patogenicidad.sh best_result_S4_sequence_corrected.fna >analisis_patogenicidad_S4.txt

Regresar al directorio del ensamblador

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}/ENSAMBLE/SPADES

## PREPARAR FASTA FINAL

Se ejecuta renombrar_fastas.sh para generar el fasta final

In [ ]:
~/analisis_influenza/bin/renombrar_fastas.sh --host NA --origin NA --year 2025